# **여행지 및 관광지 추천 인공지능**

# 데이터셋 준비

## 국내 여행로그 데이터 - AI-Hub

- 수도권: https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=&topMenu=&aihubDataSe=data&dataSetSn=71581
- 동부권: https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=&topMenu=&aihubDataSe=data&dataSetSn=71585
- 서부권: https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=&topMenu=&aihubDataSe=data&dataSetSn=71582
- 제주도 및 도서지역: https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=&topMenu=&aihubDataSe=data&dataSetSn=71584

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import pandas as pd

### 방문자 정보

In [3]:
import re

df_place = pd.concat([
    pd.read_csv('/content/gdrive/MyDrive/data/metropolitan/tn_visit_area_info_방문지정보_A.csv'),
    pd.read_csv('/content/gdrive/MyDrive/data/eastern/tn_visit_area_info_방문지정보_B.csv'),
    pd.read_csv('/content/gdrive/MyDrive/data/western/tn_visit_area_info_방문지정보_C.csv'),
    pd.read_csv('/content/gdrive/MyDrive/data/islands/tn_visit_area_info_방문지정보_D.csv')
], axis=0)

df_place

<ipython-input-3-0db7882039aa>:5: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv('/content/gdrive/MyDrive/data/eastern/tn_visit_area_info_방문지정보_B.csv'),
<ipython-input-3-0db7882039aa>:6: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv('/content/gdrive/MyDrive/data/western/tn_visit_area_info_방문지정보_C.csv'),
<ipython-input-3-0db7882039aa>:7: DtypeWarning: Columns (8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv('/content/gdrive/MyDrive/data/islands/tn_visit_area_info_방문지정보_D.csv')


,VISIT_AREA_ID,TRAVEL_ID,VISIT_ORDER,VISIT_AREA_NM,VISIT_START_YMD,VISIT_END_YMD,ROAD_NM_ADDR,LOTNO_ADDR,X_COORD,Y_COORD,...,POI_NM,RESIDENCE_TIME_MIN,VISIT_AREA_TYPE_CD,REVISIT_YN,VISIT_CHC_REASON_CD,LODGING_TYPE_CD,DGSTFN,REVISIT_INTENTION,RCMDTN_INTENTION,SGG_CD
0,2210300006,a_a015688,5,프로방스마을,2022-10-30,2022-10-30,경기 파주시 탄현면 새오리로 77,경기 파주시 탄현면 성동리 82-1,126.685239,37.790405,...,NaN,60.0,7,Y,9.0,NaN,4.0,4.0,4.0,NaN
1,2208050001,a_a000164,1,친구/친지집,2022-08-05,2022-08-05,NaN,경기도 안산시 단원구 와동,NaN,NaN,...,NaN,NaN,22,NaN,NaN,NaN,NaN,NaN,NaN,41273.0
2,2210030002,a_a004293,2,병점역 1호선,2022-10-03,2022-10-03,경기 화성시 떡전골로 97,경기 화성시 진안동 824-1,127.0332,37.206753,...,병점역,30.0,9,Y,6.0,NaN,4.0,5.0,5.0,NaN
3,2208200003,a_a000491,12,더현대서울,2022-08-20,2022-08-20,서울 영등포구 여의대로 108,서울 영등포구 여의도동 22,126.928446,37.525872,...,NaN,180.0,4,N,2.0,NaN,5.0,5.0,5.0,NaN
4,2208110007,a_a000172,13,강릉중앙시장,2022-08-11,2022-08-11,강원 강릉시 금성로 21,강원 강릉시 성남동 50,128.898624,37.754025,...,신흥수산,30.0,4,N,1.0,NaN,4.0,4.0,4.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62267,2208220008,d_d000761,8,머큐어앰배서더 제주,2022-08-22,2022-08-23,제주특별자치도 서귀포시 안덕면 한창로 365,제주특별자치도 서귀포시 안덕면 상창리 2892,126.3487373,33.29365237,...,NaN,180.0,24,N,7.0,1.0,5.0,5.0,5.0,NaN
62268,2208280007,d_d000780,2,청주국제공항,2022-08-28,2022-08-28,충북 청주시 청원구 내수읍 오창대로 980,충북 청주시 청원구 내수읍 입상리 88,127.4958529,36.72202246,...,NaN,90.0,9,Y,6.0,NaN,5.0,5.0,5.0,NaN
62269,2210030006,d_d001455,22,메종글래드제주,2022-10-03,2022-10-04,제주특별자치도 제주시 노연로 80,제주특별자치도 제주시 연동 263-15,126.4885234,33.48527907,...,메종글래드제주호텔,180.0,24,Y,8.0,1.0,3.0,3.0,3.0,NaN
62270,2209100002,a_a002144,8,옹진농협하나로마트 덕적점,2022-09-10,2022-09-10,인천 옹진군 덕적면 덕적북로 31-22,인천 옹진군 덕적면 진리 87,126.1540897,37.22679959,...,365민박,0.0,10,Y,6.0,NaN,5.0,5.0,5.0,NaN


지역별 평균 만족도를 구하기 위해 ROAD_NM_ADDR 칼럼에서 어느 지역에 속해있는지 구분하였다.

In [4]:
#df_place['SIDO_NM'] = df_place['ROAD_NM_ADDR'].str.extract(r'^(..)')
def extract_region(addr):
    region_map = {
        '서울': '서울특별시', '경기': '경기도', '충북': '충청북도', '충남': '충청남도',
        '전북': '전라북도', '전남': '전라남도', '경북': '경상북도', '경남': '경상남도',
        '강원': '강원도', '제주특별자치도': '제주특별자치도', '인천': '인천광역시', '대전': '대전광역시',
        '울산': '울산광역시', '부산': '부산광역시', '광주': '광주광역시', '대구': '대구광역시',
        '세종': '세종특별자치시'
    }

    addr = re.sub(r'\s+', ' ', addr)
    split_addr = addr.split(' ')

    if len(split_addr) >= 2:
        region = split_addr[0]
        if region in region_map:
            if '도' in region_map[region] and region != '제주특별자치도':
                return split_addr[1]
            else:
                return region_map[region]
    return 'NaN'

df_place['ROAD_NM_ADDR'] = df_place['ROAD_NM_ADDR'].astype(str)
df_place['SIDO_NM'] = df_place['ROAD_NM_ADDR'].apply(extract_region)
df_place

,VISIT_AREA_ID,TRAVEL_ID,VISIT_ORDER,VISIT_AREA_NM,VISIT_START_YMD,VISIT_END_YMD,ROAD_NM_ADDR,LOTNO_ADDR,X_COORD,Y_COORD,...,RESIDENCE_TIME_MIN,VISIT_AREA_TYPE_CD,REVISIT_YN,VISIT_CHC_REASON_CD,LODGING_TYPE_CD,DGSTFN,REVISIT_INTENTION,RCMDTN_INTENTION,SGG_CD,SIDO_NM
0,2210300006,a_a015688,5,프로방스마을,2022-10-30,2022-10-30,경기 파주시 탄현면 새오리로 77,경기 파주시 탄현면 성동리 82-1,126.685239,37.790405,...,60.0,7,Y,9.0,NaN,4.0,4.0,4.0,NaN,파주시
1,2208050001,a_a000164,1,친구/친지집,2022-08-05,2022-08-05,nan,경기도 안산시 단원구 와동,NaN,NaN,...,NaN,22,NaN,NaN,NaN,NaN,NaN,NaN,41273.0,NaN
2,2210030002,a_a004293,2,병점역 1호선,2022-10-03,2022-10-03,경기 화성시 떡전골로 97,경기 화성시 진안동 824-1,127.0332,37.206753,...,30.0,9,Y,6.0,NaN,4.0,5.0,5.0,NaN,화성시
3,2208200003,a_a000491,12,더현대서울,2022-08-20,2022-08-20,서울 영등포구 여의대로 108,서울 영등포구 여의도동 22,126.928446,37.525872,...,180.0,4,N,2.0,NaN,5.0,5.0,5.0,NaN,서울특별시
4,2208110007,a_a000172,13,강릉중앙시장,2022-08-11,2022-08-11,강원 강릉시 금성로 21,강원 강릉시 성남동 50,128.898624,37.754025,...,30.0,4,N,1.0,NaN,4.0,4.0,4.0,NaN,강릉시
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62267,2208220008,d_d000761,8,머큐어앰배서더 제주,2022-08-22,2022-08-23,제주특별자치도 서귀포시 안덕면 한창로 365,제주특별자치도 서귀포시 안덕면 상창리 2892,126.3487373,33.29365237,...,180.0,24,N,7.0,1.0,5.0,5.0,5.0,NaN,제주특별자치도
62268,2208280007,d_d000780,2,청주국제공항,2022-08-28,2022-08-28,충북 청주시 청원구 내수읍 오창대로 980,충북 청주시 청원구 내수읍 입상리 88,127.4958529,36.72202246,...,90.0,9,Y,6.0,NaN,5.0,5.0,5.0,NaN,청주시
62269,2210030006,d_d001455,22,메종글래드제주,2022-10-03,2022-10-04,제주특별자치도 제주시 노연로 80,제주특별자치도 제주시 연동 263-15,126.4885234,33.48527907,...,180.0,24,Y,8.0,1.0,3.0,3.0,3.0,NaN,제주특별자치도
62270,2209100002,a_a002144,8,옹진농협하나로마트 덕적점,2022-09-10,2022-09-10,인천 옹진군 덕적면 덕적북로 31-22,인천 옹진군 덕적면 진리 87,126.1540897,37.22679959,...,0.0,10,Y,6.0,NaN,5.0,5.0,5.0,NaN,인천광역시


### 여행 정보

In [5]:
df_travel = pd.concat([
    pd.read_csv('/content/gdrive/MyDrive/data/metropolitan/tn_travel_여행_A.csv'),
    pd.read_csv('/content/gdrive/MyDrive/data/eastern/tn_travel_여행_B.csv'),
    pd.read_csv('/content/gdrive/MyDrive/data/western/tn_travel_여행_C.csv'),
    pd.read_csv('/content/gdrive/MyDrive/data/islands/tn_travel_여행_D.csv')
], axis=0)

df_travel

,TRAVEL_ID,TRAVEL_NM,TRAVELER_ID,TRAVEL_PURPOSE,TRAVEL_START_YMD,TRAVEL_END_YMD,MVMN_NM,TRAVEL_PERSONA,TRAVEL_MISSION,TRAVEL_MISSION_CHECK
0,a_a014262,A03,a014262,1;11;2;22;,2022-10-29,2022-10-30,자가용,NaN,1;11;2;22;,2;1;22
1,a_a006728,A03,a006728,21;24;5;22;26;27;,2022-10-10,2022-10-11,자가용,NaN,21;24;5;22;26;27;,6;1;22
2,a_a015745,A02,a015745,1;2;6;,2022-10-31,2022-11-03,대중교통 등,NaN,1;2;6;,13;3;28
3,a_a010544,A03,a010544,21;22;24;26;,2022-11-01,2022-11-02,자가용,NaN,21;22;24;26;,1;22;26
4,a_a012058,A03,a012058,1;4;10;12;21;22;26;,2022-11-05,2022-11-06,대중교통 등,수도권 외 거주 39세 이하 커플,1;4;10;12;21;22;26;,1;21;22
...,...,...,...,...,...,...,...,...,...,...
3195,b_b001198,D02,b001198,1;10;3;4;5;,2022-09-04,2022-09-06,자가용,NaN,1;10;3;4;5;,25;22;21
3196,d_d009222,D01,d009222,2;,2022-10-29,2022-10-31,자가용,NaN,2;,23;2;22
3197,d_d002847,D01,d002847,21;22;23;26;28;,2022-10-16,2022-10-19,대중교통 등,NaN,21;22;23;26;28;,21;26;22
3198,d_d003147,D01,d003147,7;21;22;26;,2022-10-29,2022-10-31,자가용,NaN,7;21;22;26;,21;1;6


### 여행객 정보

In [6]:
df_traveler = pd.concat([
    pd.read_csv('/content/gdrive/MyDrive/data/metropolitan/tn_traveller_master_여행객 Master_A.csv'),
    pd.read_csv('/content/gdrive/MyDrive/data/eastern/tn_traveller_master_여행객 Master_B.csv'),
    pd.read_csv('/content/gdrive/MyDrive/data/western/tn_traveller_master_여행객 Master_C.csv'),
    pd.read_csv('/content/gdrive/MyDrive/data/islands/tn_traveller_master_여행객 Master_D.csv')
], axis=0)

df_traveler

,TRAVELER_ID,RESIDENCE_SGG_CD,GENDER,AGE_GRP,EDU_NM,EDU_FNSH_SE,MARR_STTS,FAMILY_MEMB,JOB_NM,JOB_ETC,...,TRAVEL_STYL_7,TRAVEL_STYL_8,TRAVEL_STATUS_RESIDENCE,TRAVEL_STATUS_DESTINATION,TRAVEL_STATUS_ACCOMPANY,TRAVEL_STATUS_YMD,TRAVEL_MOTIVE_1,TRAVEL_MOTIVE_2,TRAVEL_MOTIVE_3,TRAVEL_COMPANIONS_NUM
0,b015583,41,여,20,6,1.0,1.0,5,3.0,NaN,...,1,7,경기도,충남,2인 여행(가족 외),2022-10-29~2022-10-30,1.0,NaN,NaN,1
1,a001105,30,남,30,6,1.0,1.0,2,3.0,NaN,...,6,5,대전광역시,경기,3인 이상 여행(가족 외),2022-09-03~2022-09-04,3.0,NaN,NaN,7
2,a001673,41,여,30,4,1.0,2.0,3,NaN,1.0,...,2,7,경기도,서울,자녀 동반 여행,2022-09-18~2022-09-19,10.0,7.0,1.0,2
3,a017690,41,여,30,6,1.0,1.0,3,3.0,NaN,...,4,5,경기도,서울,3인 이상 여행(가족 외),2022-11-12~2022-11-12,3.0,1.0,2.0,3
4,c006666,28,여,30,6,1.0,1.0,3,13.0,NaN,...,5,5,인천광역시,경기,나홀로 여행,2022-10-18~2022-10-18,1.0,7.0,2.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3195,d002221,11,남,30,6,1.0,1.0,1,2.0,NaN,...,6,6,서울특별시,제주,2인 여행(가족 외),2022-10-02~2022-10-05,2.0,3.0,5.0,1
3196,d002109,41,여,30,5,1.0,1.0,1,3.0,NaN,...,2,7,경기도,제주,나홀로 여행,2022-10-04~2022-10-07,2.0,1.0,4.0,0
3197,d006910,50,여,30,5,1.0,1.0,1,4.0,NaN,...,6,7,제주특별자치도,제주,나홀로 여행,2022-10-18~2022-10-19,1.0,4.0,5.0,0
3198,d010634,11,여,20,6,1.0,1.0,4,3.0,NaN,...,2,7,서울특별시,제주,2인 여행(가족 외),2022-11-05~2022-11-07,2.0,5.0,1.0,1


## 전부 합치기

### 지역별 만족도를 예측하기 위한 dataframe

여행자(traveler)와 지역(SIDO_NM)을 기준으로 그룹화한 후, <br>해당 지역에서의 만족도(DGSTFN)의 평균을 계산하여 데이터프레임을 재구성하였다. <br> 이를 통해 각 여행자가 방문한 지역에서의 만족도를 파악할 수 있다.

In [7]:
df = pd.merge(df_place, df_travel, on='TRAVEL_ID', how='left')
df = pd.merge(df, df_traveler, on='TRAVELER_ID', how='left')
region_satisfaction = df.groupby(['TRAVELER_ID', 'SIDO_NM'])['DGSTFN'].mean().reset_index()

df_region = pd.merge(region_satisfaction, df_traveler, on='TRAVELER_ID', how='left')
df_region = pd.merge(df_region, df_travel, on='TRAVELER_ID', how='left')
df_region['DGSTFN'] = df_region['DGSTFN'].round(1)
df_region

,TRAVELER_ID,SIDO_NM,DGSTFN,RESIDENCE_SGG_CD,GENDER,AGE_GRP,EDU_NM,EDU_FNSH_SE,MARR_STTS,FAMILY_MEMB,...,TRAVEL_COMPANIONS_NUM,TRAVEL_ID,TRAVEL_NM,TRAVEL_PURPOSE,TRAVEL_START_YMD,TRAVEL_END_YMD,MVMN_NM,TRAVEL_PERSONA,TRAVEL_MISSION,TRAVEL_MISSION_CHECK
0,a000007,NaN,NaN,11.0,여,30.0,7.0,1.0,3.0,6.0,...,5.0,a_a000007,D02,10;23;24;26;7;,2022-08-19,2022-08-22,자가용,NaN,2;,2;3;10
1,a000007,인천광역시,3.6,11.0,여,30.0,7.0,1.0,3.0,6.0,...,5.0,a_a000007,D02,10;23;24;26;7;,2022-08-19,2022-08-22,자가용,NaN,2;,2;3;10
2,a000011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,a_a000011,A03,22;24;26;27;7;,2022-08-06,2022-08-08,자가용,NaN,2;,25;24;22
3,a000011,인천광역시,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,a_a000011,A03,22;24;26;27;7;,2022-08-06,2022-08-08,자가용,NaN,2;,25;24;22
4,a000011,파주시,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,a_a000011,A03,22;24;26;27;7;,2022-08-06,2022-08-08,자가용,NaN,2;,25;24;22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34613,d012489,제주특별자치도,4.0,50.0,여,30.0,4.0,1.0,2.0,6.0,...,0.0,d_d012489,D01,28;,2022-11-13,2022-11-14,NaN,NaN,28;,2;1;6
34614,d012490,NaN,5.0,50.0,여,20.0,6.0,1.0,1.0,1.0,...,0.0,d_d012490,D01,1;22;,2022-11-12,2022-11-13,NaN,NaN,1;22;,1;5;21
34615,d012491,NaN,4.0,50.0,남,30.0,6.0,1.0,2.0,2.0,...,1.0,d_d012491,D01,1;12;22;24;26;,2022-11-09,2022-11-10,NaN,NaN,1;12;22;24;26;,3;22;24
34616,d012491,제주특별자치도,4.3,50.0,남,30.0,6.0,1.0,2.0,2.0,...,1.0,d_d012491,D01,1;12;22;24;26;,2022-11-09,2022-11-10,NaN,NaN,1;12;22;24;26;,3;22;24


### 관광지별 만족도를 예측하기 위한 dataframe

In [8]:
df_dest = pd.merge(df_place, df_travel, on='TRAVEL_ID', how='left')
df_dest = pd.merge(df_dest, df_traveler, on='TRAVELER_ID', how='left')

df_dest

,VISIT_AREA_ID,TRAVEL_ID,VISIT_ORDER,VISIT_AREA_NM,VISIT_START_YMD,VISIT_END_YMD,ROAD_NM_ADDR,LOTNO_ADDR,X_COORD,Y_COORD,...,TRAVEL_STYL_7,TRAVEL_STYL_8,TRAVEL_STATUS_RESIDENCE,TRAVEL_STATUS_DESTINATION,TRAVEL_STATUS_ACCOMPANY,TRAVEL_STATUS_YMD,TRAVEL_MOTIVE_1,TRAVEL_MOTIVE_2,TRAVEL_MOTIVE_3,TRAVEL_COMPANIONS_NUM
0,2210300006,a_a015688,5,프로방스마을,2022-10-30,2022-10-30,경기 파주시 탄현면 새오리로 77,경기 파주시 탄현면 성동리 82-1,126.685239,37.790405,...,6.0,6.0,경기도,경기,나홀로 여행,2022-10-30~2022-10-31,1.0,7.0,10.0,0.0
1,2208050001,a_a000164,1,친구/친지집,2022-08-05,2022-08-05,nan,경기도 안산시 단원구 와동,NaN,NaN,...,3.0,5.0,경기도,경기,2인 여행(가족 외),2022-08-05~2022-08-06,3.0,2.0,1.0,1.0
2,2210030002,a_a004293,2,병점역 1호선,2022-10-03,2022-10-03,경기 화성시 떡전골로 97,경기 화성시 진안동 824-1,127.0332,37.206753,...,7.0,2.0,경기도,경기,나홀로 여행,2022-10-03~2022-10-03,6.0,2.0,1.0,0.0
3,2208200003,a_a000491,12,더현대서울,2022-08-20,2022-08-20,서울 영등포구 여의대로 108,서울 영등포구 여의도동 22,126.928446,37.525872,...,5.0,7.0,울산광역시,서울,2인 가족 여행,2022-08-19~2022-08-20,1.0,3.0,7.0,1.0
4,2208110007,a_a000172,13,강릉중앙시장,2022-08-11,2022-08-11,강원 강릉시 금성로 21,강원 강릉시 성남동 50,128.898624,37.754025,...,2.0,4.0,광주광역시,서울,나홀로 여행,2022-08-10~2022-08-13,1.0,7.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164190,2208220008,d_d000761,8,머큐어앰배서더 제주,2022-08-22,2022-08-23,제주특별자치도 서귀포시 안덕면 한창로 365,제주특별자치도 서귀포시 안덕면 상창리 2892,126.3487373,33.29365237,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
164191,2208280007,d_d000780,2,청주국제공항,2022-08-28,2022-08-28,충북 청주시 청원구 내수읍 오창대로 980,충북 청주시 청원구 내수읍 입상리 88,127.4958529,36.72202246,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
164192,2210030006,d_d001455,22,메종글래드제주,2022-10-03,2022-10-04,제주특별자치도 제주시 노연로 80,제주특별자치도 제주시 연동 263-15,126.4885234,33.48527907,...,2.0,3.0,경기도,제주,2인 가족 여행,2022-10-01~2022-10-04,1.0,3.0,2.0,1.0
164193,2209100002,a_a002144,8,옹진농협하나로마트 덕적점,2022-09-10,2022-09-10,인천 옹진군 덕적면 덕적북로 31-22,인천 옹진군 덕적면 진리 87,126.1540897,37.22679959,...,3.0,7.0,서울특별시,인천,2인 여행(가족 외),2022-09-09~2022-09-10,1.0,3.0,7.0,1.0


# 데이터 셋 전처리

## 데이터 셋 전처리1

### 지역별 만족도 필터

In [9]:
df_filter = df_region[~df_region['TRAVEL_MISSION_CHECK'].isnull()].copy()
df_filter.loc[:, 'TRAVEL_MISSION_INT'] = df_filter['TRAVEL_MISSION_CHECK'].str.split(';').str[0].astype(int)

df_filter

,TRAVELER_ID,SIDO_NM,DGSTFN,RESIDENCE_SGG_CD,GENDER,AGE_GRP,EDU_NM,EDU_FNSH_SE,MARR_STTS,FAMILY_MEMB,...,TRAVEL_ID,TRAVEL_NM,TRAVEL_PURPOSE,TRAVEL_START_YMD,TRAVEL_END_YMD,MVMN_NM,TRAVEL_PERSONA,TRAVEL_MISSION,TRAVEL_MISSION_CHECK,TRAVEL_MISSION_INT
0,a000007,NaN,NaN,11.0,여,30.0,7.0,1.0,3.0,6.0,...,a_a000007,D02,10;23;24;26;7;,2022-08-19,2022-08-22,자가용,NaN,2;,2;3;10,2
1,a000007,인천광역시,3.6,11.0,여,30.0,7.0,1.0,3.0,6.0,...,a_a000007,D02,10;23;24;26;7;,2022-08-19,2022-08-22,자가용,NaN,2;,2;3;10,2
2,a000011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,a_a000011,A03,22;24;26;27;7;,2022-08-06,2022-08-08,자가용,NaN,2;,25;24;22,25
3,a000011,인천광역시,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,a_a000011,A03,22;24;26;27;7;,2022-08-06,2022-08-08,자가용,NaN,2;,25;24;22,25
4,a000011,파주시,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,a_a000011,A03,22;24;26;27;7;,2022-08-06,2022-08-08,자가용,NaN,2;,25;24;22,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34613,d012489,제주특별자치도,4.0,50.0,여,30.0,4.0,1.0,2.0,6.0,...,d_d012489,D01,28;,2022-11-13,2022-11-14,NaN,NaN,28;,2;1;6,2
34614,d012490,NaN,5.0,50.0,여,20.0,6.0,1.0,1.0,1.0,...,d_d012490,D01,1;22;,2022-11-12,2022-11-13,NaN,NaN,1;22;,1;5;21,1
34615,d012491,NaN,4.0,50.0,남,30.0,6.0,1.0,2.0,2.0,...,d_d012491,D01,1;12;22;24;26;,2022-11-09,2022-11-10,NaN,NaN,1;12;22;24;26;,3;22;24,3
34616,d012491,제주특별자치도,4.3,50.0,남,30.0,6.0,1.0,2.0,2.0,...,d_d012491,D01,1;12;22;24;26;,2022-11-09,2022-11-10,NaN,NaN,1;12;22;24;26;,3;22;24,3


### 관광지별 만족도 필터

In [10]:
df_filter2 = df_dest[~df_dest['TRAVEL_MISSION_CHECK'].isnull()].copy()
df_filter2.loc[:, 'TRAVEL_MISSION_INT'] = df_filter2['TRAVEL_MISSION_CHECK'].str.split(';').str[0].astype(int)

df_filter2

,VISIT_AREA_ID,TRAVEL_ID,VISIT_ORDER,VISIT_AREA_NM,VISIT_START_YMD,VISIT_END_YMD,ROAD_NM_ADDR,LOTNO_ADDR,X_COORD,Y_COORD,...,TRAVEL_STYL_8,TRAVEL_STATUS_RESIDENCE,TRAVEL_STATUS_DESTINATION,TRAVEL_STATUS_ACCOMPANY,TRAVEL_STATUS_YMD,TRAVEL_MOTIVE_1,TRAVEL_MOTIVE_2,TRAVEL_MOTIVE_3,TRAVEL_COMPANIONS_NUM,TRAVEL_MISSION_INT
0,2210300006,a_a015688,5,프로방스마을,2022-10-30,2022-10-30,경기 파주시 탄현면 새오리로 77,경기 파주시 탄현면 성동리 82-1,126.685239,37.790405,...,6.0,경기도,경기,나홀로 여행,2022-10-30~2022-10-31,1.0,7.0,10.0,0.0,5
1,2208050001,a_a000164,1,친구/친지집,2022-08-05,2022-08-05,nan,경기도 안산시 단원구 와동,NaN,NaN,...,5.0,경기도,경기,2인 여행(가족 외),2022-08-05~2022-08-06,3.0,2.0,1.0,1.0,22
2,2210030002,a_a004293,2,병점역 1호선,2022-10-03,2022-10-03,경기 화성시 떡전골로 97,경기 화성시 진안동 824-1,127.0332,37.206753,...,2.0,경기도,경기,나홀로 여행,2022-10-03~2022-10-03,6.0,2.0,1.0,0.0,28
3,2208200003,a_a000491,12,더현대서울,2022-08-20,2022-08-20,서울 영등포구 여의대로 108,서울 영등포구 여의도동 22,126.928446,37.525872,...,7.0,울산광역시,서울,2인 가족 여행,2022-08-19~2022-08-20,1.0,3.0,7.0,1.0,6
4,2208110007,a_a000172,13,강릉중앙시장,2022-08-11,2022-08-11,강원 강릉시 금성로 21,강원 강릉시 성남동 50,128.898624,37.754025,...,4.0,광주광역시,서울,나홀로 여행,2022-08-10~2022-08-13,1.0,7.0,2.0,0.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164189,2209270001,d_d001473,12,문개항아리 조천본점,2022-09-27,2022-09-27,제주특별자치도 제주시 조천읍 조함해안로 217-1,제주특별자치도 제주시 조천읍 신흥리 792-1,126.6440146,33.55347531,...,6.0,경기도,제주,2인 가족 여행,2022-09-26~2022-09-28,2.0,3.0,9.0,1.0,26
164190,2208220008,d_d000761,8,머큐어앰배서더 제주,2022-08-22,2022-08-23,제주특별자치도 서귀포시 안덕면 한창로 365,제주특별자치도 서귀포시 안덕면 상창리 2892,126.3487373,33.29365237,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23
164192,2210030006,d_d001455,22,메종글래드제주,2022-10-03,2022-10-04,제주특별자치도 제주시 노연로 80,제주특별자치도 제주시 연동 263-15,126.4885234,33.48527907,...,3.0,경기도,제주,2인 가족 여행,2022-10-01~2022-10-04,1.0,3.0,2.0,1.0,2
164193,2209100002,a_a002144,8,옹진농협하나로마트 덕적점,2022-09-10,2022-09-10,인천 옹진군 덕적면 덕적북로 31-22,인천 옹진군 덕적면 진리 87,126.1540897,37.22679959,...,7.0,서울특별시,인천,2인 여행(가족 외),2022-09-09~2022-09-10,1.0,3.0,7.0,1.0,24


## 데이터 셋 전처리2
- 필요한 열만 추출
- 비어있는 행 삭제

### 지역별 만족도 필터

In [11]:
df_filter = df_filter[[
    'GENDER',
    'AGE_GRP',
    'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3', 'TRAVEL_STYL_4', 'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 'TRAVEL_STYL_7', 'TRAVEL_STYL_8',
    'TRAVEL_MOTIVE_1',
    'TRAVEL_COMPANIONS_NUM',
    'TRAVEL_MISSION_INT',
    #'VISIT_AREA_NM',
    'DGSTFN',
    'SIDO_NM'
]]

df_filter = df_filter.dropna()

df_filter

,GENDER,AGE_GRP,TRAVEL_STYL_1,TRAVEL_STYL_2,TRAVEL_STYL_3,TRAVEL_STYL_4,TRAVEL_STYL_5,TRAVEL_STYL_6,TRAVEL_STYL_7,TRAVEL_STYL_8,TRAVEL_MOTIVE_1,TRAVEL_COMPANIONS_NUM,TRAVEL_MISSION_INT,DGSTFN,SIDO_NM
1,여,30.0,3.0,1.0,1.0,1.0,1.0,3.0,2.0,6.0,1.0,5.0,2,3.6,인천광역시
5,남,30.0,3.0,1.0,1.0,4.0,3.0,4.0,1.0,5.0,8.0,0.0,3,4.2,NaN
6,남,30.0,3.0,1.0,1.0,4.0,3.0,4.0,1.0,5.0,8.0,0.0,3,3.7,수원시
11,남,50.0,2.0,2.0,6.0,2.0,2.0,6.0,2.0,7.0,10.0,7.0,21,3.0,고양시
12,남,50.0,2.0,2.0,6.0,2.0,2.0,6.0,2.0,7.0,10.0,7.0,21,4.0,양평군
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34612,여,30.0,6.0,6.0,2.0,2.0,2.0,6.0,6.0,2.0,6.0,0.0,2,2.0,NaN
34613,여,30.0,6.0,6.0,2.0,2.0,2.0,6.0,6.0,2.0,6.0,0.0,2,4.0,제주특별자치도
34614,여,20.0,1.0,1.0,1.0,4.0,4.0,2.0,7.0,1.0,1.0,0.0,1,5.0,NaN
34615,남,30.0,4.0,1.0,2.0,4.0,6.0,6.0,6.0,7.0,7.0,1.0,3,4.0,NaN


### 관광지별 만족도 필터

In [12]:
df_filter2 = df_filter2[[
    'GENDER',
    'AGE_GRP',
    'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3', 'TRAVEL_STYL_4', 'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 'TRAVEL_STYL_7', 'TRAVEL_STYL_8',
    'TRAVEL_MOTIVE_1',
    'TRAVEL_COMPANIONS_NUM',
    'TRAVEL_MISSION_INT',
    'VISIT_AREA_TYPE_CD',
    'VISIT_AREA_NM',
    'DGSTFN',
    'SIDO_NM'
]]

df_filter2 = df_filter2.dropna()

df_filter2

,GENDER,AGE_GRP,TRAVEL_STYL_1,TRAVEL_STYL_2,TRAVEL_STYL_3,TRAVEL_STYL_4,TRAVEL_STYL_5,TRAVEL_STYL_6,TRAVEL_STYL_7,TRAVEL_STYL_8,TRAVEL_MOTIVE_1,TRAVEL_COMPANIONS_NUM,TRAVEL_MISSION_INT,VISIT_AREA_TYPE_CD,VISIT_AREA_NM,DGSTFN,SIDO_NM
0,남,30.0,4.0,4.0,2.0,4.0,6.0,5.0,6.0,6.0,1.0,0.0,5,7,프로방스마을,4.0,파주시
2,여,40.0,4.0,3.0,6.0,3.0,2.0,2.0,7.0,2.0,6.0,0.0,28,9,병점역 1호선,4.0,화성시
3,여,30.0,4.0,4.0,4.0,3.0,4.0,4.0,5.0,7.0,1.0,1.0,6,4,더현대서울,5.0,서울특별시
4,남,30.0,7.0,1.0,1.0,1.0,7.0,7.0,2.0,4.0,1.0,0.0,2,4,강릉중앙시장,4.0,강릉시
5,남,20.0,6.0,1.0,1.0,3.0,4.0,6.0,3.0,2.0,3.0,1.0,11,7,청계천,4.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164187,여,20.0,1.0,2.0,2.0,4.0,5.0,2.0,5.0,6.0,1.0,2.0,28,11,함덕촐래밥상,5.0,제주특별자치도
164189,남,30.0,3.0,3.0,3.0,4.0,3.0,3.0,4.0,6.0,2.0,1.0,26,11,문개항아리 조천본점,5.0,제주특별자치도
164192,남,40.0,2.0,2.0,5.0,1.0,1.0,3.0,2.0,3.0,1.0,1.0,2,24,메종글래드제주,3.0,제주특별자치도
164193,여,30.0,1.0,1.0,1.0,3.0,1.0,1.0,3.0,7.0,1.0,1.0,24,10,옹진농협하나로마트 덕적점,5.0,인천광역시


## 데이터 셋 전처리3

- 범주형(categorical) 데이터 정의
- 범주형 데이터는 string or integer 형태여야 함

### 지역별 카테고리 특징

In [13]:
categorical_features_names = [
    'GENDER',
    #'AGE_GRP',
    'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3', 'TRAVEL_STYL_4', 'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 'TRAVEL_STYL_7', 'TRAVEL_STYL_8',
    'TRAVEL_MOTIVE_1',
    #'TRAVEL_COMPANIONS_NUM',
    'TRAVEL_MISSION_INT',
    #'VISIT_AREA_NM',
    'SIDO_NM'
    #'DGSTFN',
]

df_filter[categorical_features_names[1:-1]] = df_filter[categorical_features_names[1:-1]].astype(int)
df_filter

,GENDER,AGE_GRP,TRAVEL_STYL_1,TRAVEL_STYL_2,TRAVEL_STYL_3,TRAVEL_STYL_4,TRAVEL_STYL_5,TRAVEL_STYL_6,TRAVEL_STYL_7,TRAVEL_STYL_8,TRAVEL_MOTIVE_1,TRAVEL_COMPANIONS_NUM,TRAVEL_MISSION_INT,DGSTFN,SIDO_NM
1,여,30.0,3,1,1,1,1,3,2,6,1,5.0,2,3.6,인천광역시
5,남,30.0,3,1,1,4,3,4,1,5,8,0.0,3,4.2,NaN
6,남,30.0,3,1,1,4,3,4,1,5,8,0.0,3,3.7,수원시
11,남,50.0,2,2,6,2,2,6,2,7,10,7.0,21,3.0,고양시
12,남,50.0,2,2,6,2,2,6,2,7,10,7.0,21,4.0,양평군
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34612,여,30.0,6,6,2,2,2,6,6,2,6,0.0,2,2.0,NaN
34613,여,30.0,6,6,2,2,2,6,6,2,6,0.0,2,4.0,제주특별자치도
34614,여,20.0,1,1,1,4,4,2,7,1,1,0.0,1,5.0,NaN
34615,남,30.0,4,1,2,4,6,6,6,7,7,1.0,3,4.0,NaN


### 관광지별 카테고리 특징

In [14]:
categorical_features_names2 = [
    'GENDER',
    #'AGE_GRP',
    'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3', 'TRAVEL_STYL_4', 'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 'TRAVEL_STYL_7', 'TRAVEL_STYL_8',
    'TRAVEL_MOTIVE_1',
    #'TRAVEL_COMPANIONS_NUM',
    'TRAVEL_MISSION_INT',
    'VISIT_AREA_TYPE_CD',
    'VISIT_AREA_NM',
    'SIDO_NM'
    #'DGSTFN',
]

df_filter2[categorical_features_names2[1:-2]] = df_filter2[categorical_features_names2[1:-2]].astype(int)
df_filter2

,GENDER,AGE_GRP,TRAVEL_STYL_1,TRAVEL_STYL_2,TRAVEL_STYL_3,TRAVEL_STYL_4,TRAVEL_STYL_5,TRAVEL_STYL_6,TRAVEL_STYL_7,TRAVEL_STYL_8,TRAVEL_MOTIVE_1,TRAVEL_COMPANIONS_NUM,TRAVEL_MISSION_INT,VISIT_AREA_TYPE_CD,VISIT_AREA_NM,DGSTFN,SIDO_NM
0,남,30.0,4,4,2,4,6,5,6,6,1,0.0,5,7,프로방스마을,4.0,파주시
2,여,40.0,4,3,6,3,2,2,7,2,6,0.0,28,9,병점역 1호선,4.0,화성시
3,여,30.0,4,4,4,3,4,4,5,7,1,1.0,6,4,더현대서울,5.0,서울특별시
4,남,30.0,7,1,1,1,7,7,2,4,1,0.0,2,4,강릉중앙시장,4.0,강릉시
5,남,20.0,6,1,1,3,4,6,3,2,3,1.0,11,7,청계천,4.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164187,여,20.0,1,2,2,4,5,2,5,6,1,2.0,28,11,함덕촐래밥상,5.0,제주특별자치도
164189,남,30.0,3,3,3,4,3,3,4,6,2,1.0,26,11,문개항아리 조천본점,5.0,제주특별자치도
164192,남,40.0,2,2,5,1,1,3,2,3,1,1.0,2,24,메종글래드제주,3.0,제주특별자치도
164193,여,30.0,1,1,1,3,1,1,3,7,1,1.0,24,10,옹진농협하나로마트 덕적점,5.0,인천광역시


## 데이터셋 전처리 4

- 학습/테스트 데이터 나누기

In [16]:
from sklearn.model_selection import train_test_split

### 지역별 학습 및 테스트 데이터

In [17]:
train_data, test_data =  train_test_split(df_filter, test_size=0.2, random_state=42)

print(train_data.shape)
print(test_data.shape)

(19245, 15)
(4812, 15)


###  관광지별 학습 및 테스트 데이터

In [18]:
train_data2, test_data2 = train_test_split(df_filter2, test_size=0.2, random_state=42)

print(train_data2.shape)
print(test_data2.shape)

(69526, 17)
(17382, 17)


# CatBoost

In [26]:
!pip install -q catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 2.2 MB/s eta 0:00:00


In [27]:
from catboost import CatBoostRegressor, Pool
import numpy as np

## 데이터 형태 정의

- 데이터의 만족도가 상향평준화 된 경향이 있음
- 만족도가 2 이하이면 가중치를 2 부여하고 아니면 1 부여하도록 설정

### 지역별 만족도 예측 모델의 데이터 형태 정의

In [28]:
weights = np.where(train_data['DGSTFN'] <= 2, 2, 1)

train_pool = Pool(train_data.drop(['DGSTFN'], axis=1),
                  label=train_data['DGSTFN'],
                  cat_features=categorical_features_names,
                  weight=weights
                  )

test_pool = Pool(test_data.drop(['DGSTFN'], axis=1),
                 label=test_data['DGSTFN'],
                 cat_features=categorical_features_names
                 )

### 관광지별 만족도 예측 모델의 데이터 형태 정의

In [29]:
weights = np.where(train_data2['DGSTFN'] <= 2, 2, 1)

train_pool2 = Pool(train_data2.drop(['DGSTFN'], axis=1),
                  label=train_data2['DGSTFN'],
                  cat_features=categorical_features_names2,
                  weight=weights
)

test_pool2 = Pool(test_data2.drop(['DGSTFN'], axis=1),
                 label=test_data2['DGSTFN'],
                 cat_features=categorical_features_names2
                 )

## 모델 정의 및 학습

### 지역별 만족도 예측 모델

In [30]:
model = CatBoostRegressor(
    loss_function='RMSE',
    eval_metric='RMSE',
    task_type='GPU',
    depth=8,
    learning_rate=0.01,
    n_estimators=2000)

model.fit(
    train_pool,
    eval_set=test_pool,
    verbose=500,
    plot=True)


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.7664163	test: 0.6901653	best: 0.6901653 (0)	total: 328ms	remaining: 10m 54s
500:	learn: 0.7397053	test: 0.6748626	best: 0.6748626 (500)	total: 54.1s	remaining: 2m 41s
1000:	learn: 0.7289417	test: 0.6705510	best: 0.6705510 (1000)	total: 1m 26s	remaining: 1m 26s
1500:	learn: 0.7218349	test: 0.6691084	best: 0.6690935 (1495)	total: 2m 4s	remaining: 41.2s
1999:	learn: 0.7161515	test: 0.6679882	best: 0.6679882 (1999)	total: 2m 36s	remaining: 0us
bestTest = 0.6679882418
bestIteration = 1999


In [31]:
model.save_model('/content/gdrive/MyDrive/region_recommendation.cbm')

### 관광지별 만족도 예측 모델

In [32]:
model2 = CatBoostRegressor(
    loss_function='RMSE',
    eval_metric='RMSE',
    task_type='GPU',
    depth=8,
    learning_rate=0.01,
    n_estimators=2000
)

model2.fit(
    train_pool2,
    eval_set=test_pool2,
    verbose=500,
    plot=True
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.9528408	test: 0.8529238	best: 0.8529238 (0)	total: 60.8ms	remaining: 2m 1s
500:	learn: 0.9045759	test: 0.8084547	best: 0.8084547 (500)	total: 18s	remaining: 54s
1000:	learn: 0.8869129	test: 0.7977948	best: 0.7977948 (1000)	total: 35.7s	remaining: 35.6s
1500:	learn: 0.8752394	test: 0.7938002	best: 0.7937983 (1498)	total: 54s	remaining: 18s
1999:	learn: 0.8647045	test: 0.7915552	best: 0.7915552 (1999)	total: 1m 16s	remaining: 0us
bestTest = 0.7915551929
bestIteration = 1999


In [33]:
model2.save_model('/content/gdrive/MyDrive/place_recommendation.cbm')

# 테스트

## 주요 특징

In [34]:
model.get_feature_importance(prettified=True)

,Feature Id,Importances
0,TRAVEL_STYL_8,10.120193
1,SIDO_NM,10.029177
2,TRAVEL_MISSION_INT,9.300557
3,TRAVEL_STYL_2,8.941030
4,TRAVEL_STYL_3,8.254528
5,TRAVEL_STYL_4,8.253412
6,TRAVEL_MOTIVE_1,8.093476
7,TRAVEL_STYL_5,7.444938
8,TRAVEL_STYL_6,7.299931
9,TRAVEL_STYL_7,7.208609


In [35]:
model2.get_feature_importance(prettified=True)

,Feature Id,Importances
0,VISIT_AREA_TYPE_CD,10.297925
1,TRAVEL_MISSION_INT,10.083020
2,TRAVEL_STYL_8,9.387393
3,TRAVEL_STYL_4,7.153968
4,SIDO_NM,6.484321
5,TRAVEL_STYL_5,6.416239
6,VISIT_AREA_NM,6.407344
7,TRAVEL_STYL_6,6.225661
8,TRAVEL_STYL_7,6.111695
9,TRAVEL_STYL_1,6.004228


## 특정 데이터의 실제 만족도와 예측 만족도

지역별 만족도 예측

In [36]:
test_data.iloc[1].to_dict()

{'GENDER': '여',
 'AGE_GRP': 40.0,
 'TRAVEL_STYL_1': 2,
 'TRAVEL_STYL_2': 2,
 'TRAVEL_STYL_3': 6,
 'TRAVEL_STYL_4': 4,
 'TRAVEL_STYL_5': 2,
 'TRAVEL_STYL_6': 2,
 'TRAVEL_STYL_7': 6,
 'TRAVEL_STYL_8': 3,
 'TRAVEL_MOTIVE_1': 4,
 'TRAVEL_COMPANIONS_NUM': 0.0,
 'TRAVEL_MISSION_INT': 9,
 'DGSTFN': 4.5,
 'SIDO_NM': '수원시'}

In [37]:
model.predict(test_data.iloc[1].drop(['DGSTFN']))

4.103676101636641

관광지별 만족도 예측

In [38]:
test_data2.iloc[102].to_dict()

{'GENDER': '남',
 'AGE_GRP': 20.0,
 'TRAVEL_STYL_1': 4,
 'TRAVEL_STYL_2': 1,
 'TRAVEL_STYL_3': 1,
 'TRAVEL_STYL_4': 4,
 'TRAVEL_STYL_5': 6,
 'TRAVEL_STYL_6': 3,
 'TRAVEL_STYL_7': 2,
 'TRAVEL_STYL_8': 5,
 'TRAVEL_MOTIVE_1': 2,
 'TRAVEL_COMPANIONS_NUM': 1.0,
 'TRAVEL_MISSION_INT': 9,
 'VISIT_AREA_TYPE_CD': 11,
 'VISIT_AREA_NM': '어썸 경성대부경대점',
 'DGSTFN': 4.0,
 'SIDO_NM': '부산광역시'}

In [39]:
test_data2.iloc[102].drop('DGSTFN')

GENDER                            남
AGE_GRP                        20.0
TRAVEL_STYL_1                     4
TRAVEL_STYL_2                     1
TRAVEL_STYL_3                     1
TRAVEL_STYL_4                     4
TRAVEL_STYL_5                     6
TRAVEL_STYL_6                     3
TRAVEL_STYL_7                     2
TRAVEL_STYL_8                     5
TRAVEL_MOTIVE_1                   2
TRAVEL_COMPANIONS_NUM           1.0
TRAVEL_MISSION_INT                9
VISIT_AREA_TYPE_CD               11
VISIT_AREA_NM            어썸 경성대부경대점
SIDO_NM                       부산광역시
Name: 57933, dtype: object

In [40]:
model2.predict(test_data2.iloc[102].drop('DGSTFN'))

4.388213181818259

## 만족도가 높을 것으로 예상되는 지역 10개 출력

In [41]:
regions = df_filter[['SIDO_NM']].dropna().drop_duplicates()

regions

,SIDO_NM
1,인천광역시
5,NaN
6,수원시
11,고양시
12,양평군
...,...
18795,화순군
18859,무안군
18928,영광군
20728,순창군


In [42]:
traveler = {
'GENDER': '여',
 'AGE_GRP': 30.0,
 'TRAVEL_STYL_1': 1,
 'TRAVEL_STYL_2': 1,
 'TRAVEL_STYL_3': 6,
 'TRAVEL_STYL_4': 5,
 'TRAVEL_STYL_5': 3,
 'TRAVEL_STYL_6': 4,
 'TRAVEL_STYL_7': 4,
 'TRAVEL_STYL_8': 5,
 'TRAVEL_MOTIVE_1': 3,
 'TRAVEL_COMPANIONS_NUM': 6.0,
 'TRAVEL_MISSION_INT': 10,
}

results = pd.DataFrame([], columns=['REGION', 'SCORE'])

for region in regions['SIDO_NM']:
    if region == 'NaN':
        continue

    input = list(traveler.values())
    input.append(region)

    score = model.predict(input)

    results = pd.concat([results, pd.DataFrame([[region, score]], columns=['REGION', 'SCORE'])])

results.sort_values('SCORE', ascending=False)[:10]

,REGION,SCORE
0,대전광역시,4.299520
0,부산광역시,4.299513
0,제주특별자치도,4.294459
0,서울특별시,4.288494
0,대구광역시,4.285275
0,인천광역시,4.283540
0,경주시,4.282911
0,전주시,4.274883
0,춘천시,4.270481
0,강릉시,4.269425


### 서울특별시에서 만족도가 가장 높을 것으로 예상되는 관광지 10개 출력

In [43]:
places = df_filter2.loc[(df_filter2['SIDO_NM'] == '서울특별시') &
                        (df_filter2['VISIT_AREA_TYPE_CD'].isin([1, 2, 3, 6, 7, 13])),
                        ['VISIT_AREA_NM']].dropna().drop_duplicates()
places

,VISIT_AREA_NM
24,더현대서울
25,경복궁
36,아모레퍼시픽미술관
38,세운전자상가
88,남산공원
...,...
88165,매헌윤봉길의사기념관
92544,메가박스 목동
94593,BMW 도이치모터스 양재전시장
142972,연화사


In [44]:
traveler2 = {
 'GENDER': '남',
 'AGE_GRP': 20.0,
 'TRAVEL_STYL_1': 4,
 'TRAVEL_STYL_2': 1,
 'TRAVEL_STYL_3': 1,
 'TRAVEL_STYL_4': 4,
 'TRAVEL_STYL_5': 6,
 'TRAVEL_STYL_6': 3,
 'TRAVEL_STYL_7': 2,
 'TRAVEL_STYL_8': 5,
 'TRAVEL_MOTIVE_1': 2,
 'TRAVEL_COMPANIONS_NUM': 1.0,
 'TRAVEL_MISSION_INT': 9,
 'VISIT_AREA_TYPE_CD': 1,
 'VISIT_AREA_NM': '어썸 경성대부경대점',
 'SIDO_NM': '서울특별시'
}

results2 = pd.DataFrame([], columns=['PLACE', 'SCORE'])

for place in places['VISIT_AREA_NM']:
    if place == 'NaN':
        continue

    traveler2['VISIT_AREA_NM'] = place
    input2 = list(traveler2.values())

    score = model2.predict(input2)

    results2 = pd.concat([results2, pd.DataFrame([[place, score]], columns=['PLACE', 'SCORE'])])

results2.sort_values('SCORE', ascending=False)[:10]

,PLACE,SCORE
0,서울역,4.691117
0,경복궁,4.680564
0,더현대서울,4.665215
0,창경궁,4.664447
0,여의도한강공원,4.663951
0,국립중앙박물관,4.661027
0,창덕궁,4.658015
0,동대문디자인플라자,4.653454
0,올림픽공원 88잔디마당,4.652735
0,동대문디자인플라자(DDP) 동대문역사문화공원,4.652038


In [45]:
df_place.sort_values(by='VISIT_AREA_TYPE_CD', inplace=True)

for value in df_place['VISIT_AREA_TYPE_CD'].unique():
    first_value = df_place.loc[df_place['VISIT_AREA_TYPE_CD'] == value, 'VISIT_AREA_NM'].iloc[20]
    print(f"VISIT_AREA_TYPE_CD: {value}, VISIT_AREA_NM: {first_value}")

VISIT_AREA_TYPE_CD: 1, VISIT_AREA_NM: 금능해수욕장
VISIT_AREA_TYPE_CD: 2, VISIT_AREA_NM: 제주항공우주박물관
VISIT_AREA_TYPE_CD: 3, VISIT_AREA_NM: 메가박스 정관
VISIT_AREA_TYPE_CD: 4, VISIT_AREA_NM: 이마트 남양주점
VISIT_AREA_TYPE_CD: 5, VISIT_AREA_NM: 윈드1947카트테마파크
VISIT_AREA_TYPE_CD: 6, VISIT_AREA_NM: 엑스포과학공원
VISIT_AREA_TYPE_CD: 7, VISIT_AREA_NM: 용담삼동 2159
VISIT_AREA_TYPE_CD: 8, VISIT_AREA_NM: 수원화성 미디어아트쇼
VISIT_AREA_TYPE_CD: 9, VISIT_AREA_NM: 제주국제공항
VISIT_AREA_TYPE_CD: 10, VISIT_AREA_NM: 가마로강정 광주충파점
VISIT_AREA_TYPE_CD: 11, VISIT_AREA_NM: 정돈 갤러리아 광교점
VISIT_AREA_TYPE_CD: 12, VISIT_AREA_NM: 한성미인
VISIT_AREA_TYPE_CD: 13, VISIT_AREA_NM: 몽토랑산양목장
VISIT_AREA_TYPE_CD: 21, VISIT_AREA_NM: 집
VISIT_AREA_TYPE_CD: 22, VISIT_AREA_NM: 한성필하우스아파트
VISIT_AREA_TYPE_CD: 23, VISIT_AREA_NM: 사무실
VISIT_AREA_TYPE_CD: 24, VISIT_AREA_NM: 썸무인텔


필요한 csv 파일 다운로드

In [46]:
df_region['SIDO_NM'].drop_duplicates().to_csv('/content/gdrive/MyDrive/regions.csv')

In [ ]:
df_place_unique = df_place[['SIDO_NM', 'VISIT_AREA_NM', 'VISIT_AREA_TYPE_CD']].copy()
df_place_unique.drop_duplicates(subset='VISIT_AREA_NM', inplace=True)
df_place_unique.to_csv('/content/gdrive/MyDrive/places.csv', index=False)